In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
%matplotlib inline

### Join dataset

#### Download

Download all four datasets from here: https://crtm.maps.arcgis.com/apps/MinimalGallery/index.html?appid=a60bb2f0142b440eadee1a69a11693fc
and store in *data\raw\*

**Data**

In [4]:
ind = pd.read_excel (r'..\data\raw\EDM2018INDIVIDUOS.xlsx')
hog = pd.read_excel (r'..\data\raw\EDM2018HOGARES.xlsx')
via = pd.read_excel (r'..\data\raw\EDM2018VIAJES.xlsx', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})
etap = pd.read_excel (r'..\data\raw\EDM2018XETAPAS.xlsx')

via.set_index(["ID_HOGAR", "ID_IND", "ID_VIAJE"], inplace = True)
ind.set_index(["ID_HOGAR", "ID_IND"], inplace = True)
hog.set_index("ID_HOGAR", inplace = True)
etap.set_index(["ID_HOGAR", "ID_IND", "ID_VIAJE"], inplace =True)

In [5]:
people = hog.join(ind, lsuffix = "_hog", rsuffix = "_ind")
trips = people.join(via, rsuffix = "_via")
legs = trips.join(etap, rsuffix = "_etap")

people.to_csv(r'..\data\interim\people.csv')
trips.to_csv(r'..\data\interim\trips.csv')
legs.to_csv(r'..\data\interim\legs.csv')

**Codes**

In [7]:
codes_hog = pd.read_excel (r'..\data\raw\EDM2018HOGARES.xlsx', sheet_name = 1)
codes_ind = pd.read_excel (r'..\data\raw\EDM2018INDIVIDUOS.xlsx', sheet_name = 1)
codes_via = pd.read_excel (r'..\data\raw\EDM2018VIAJES.xlsx', sheet_name = 1)
codes_eta = pd.read_excel (r'..\data\raw\EDM2018XETAPAS.xlsx', sheet_name = 1)

codes = codes_hog.append(codes_ind, ignore_index = True, sort = False
                        ).append(codes_via, ignore_index = True, sort = False
                                ).append(codes_eta, ignore_index = True, sort = False
                                        ).drop(columns = ["Unnamed: 0", "Unnamed: 1"])

codes["VARIABLE"] = codes.VARIABLE.fillna(method = "ffill")
codes["ESPECIFICACIÓN"] = codes["ESPECIFICACIÓN"].fillna(method = "ffill")

codes['CODE'], codes['VALUE'] = codes['VALORES'].str.split(".", 1).str
codes['CODE'], unused = codes.CODE.str.split(" '", 1).str

codes.drop(index = codes[codes.VARIABLE.isna()].index, inplace = True)

codes.to_csv(r'..\data\interim\codes.csv', index = False)

<ipython-input-7-d8297db5a64f>:14: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  codes['CODE'], codes['VALUE'] = codes['VALORES'].str.split(".", 1).str
<ipython-input-7-d8297db5a64f>:15: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  codes['CODE'], unused = codes.CODE.str.split(" '", 1).str


translated codes not programmatically, but manually via google Sheets.

- Translation is stored in data\processed\codes_translated.csv